In [1]:
using CSV, DataFrames
using Statistics, HypothesisTests
using Plots
using StatsPlots

In [2]:
df_x = DataFrame(CSV.File("weights/zcta_X.csv"))[:, ["ZIP", "1"]]
rename!(df_x, [("1" => :x)]);

In [6]:
bhd = DataFrame(CSV.File("raw_data/cdc_places/PLACES__Local_Data_for_Better_Health__ZCTA_Data_2024_release_20250523.csv"))

m = [v == "Crude prevalence" for v in bhd[:, "Data_Value_Type"]]
bhd = bhd[m,:]

m = [v == 2022 for v in bhd[:, "Year"]]
bhd = bhd[m,:]

l = [
    "Frequent physical distress among adults",
    "Stroke among adults",
    "Feeling socially isolated among adults",
    "Fair or poor self-rated health status among adults",
    "All teeth lost among adults aged >=65 years",
    "Vision disability among adults",
    "Cognitive disability among adults",
    "Frequent mental distress among adults",
    "Lack of social and emotional support among adults",
    "Mobility disability among adults",
    "Current asthma among adults",
    "Chronic obstructive pulmonary disease among adults",
    "Coronary heart disease among adults",
    "Diagnosed diabetes among adults",
    "Hearing disability among adults",
    "Depression among adults",
    "Short sleep duration among adults",
    "Obesity among adults",
]

bhd_t = unique(bhd[:, [:LocationID]])
dropmissing!(bhd_t)

for (i, v) in enumerate(l)
    m = bhd[:, :Measure] .== v
    y = bhd[m, [:LocationID, :Data_Value]]
    y = combine(groupby(y, :LocationID), :Data_Value=>first=>:Data_Value)
    dropmissing!(y)
    bhd_t = leftjoin(bhd_t, y, on=:LocationID)
    rename!(bhd_t, [(:Data_Value => v)])
end

In [7]:
rename!(bhd_t, [(:LocationID => :ZIP)]);

res = outerjoin(bhd_t, df_x, on=:ZIP);

In [8]:
res[:, "x"] = coalesce.(res[:, "x"], NaN)
vals = zeros(size(l))
for (i, v) in enumerate(l)
    res[:, v] = coalesce.(res[:, v], NaN)
    m = (isnan.(res[:, v]) .+ isnan.(res[:, "x"])) .== 0
    vals[i] = cor(res[m, "x"], res[m, v])
    println(cor(res[m, "x"], res[m, v]), "  ", v)
end

0.5982094125074141  Frequent physical distress among adults
0.5455413188691648  Stroke among adults
0.25079458773391056  Feeling socially isolated among adults
0.6014420902543405  Fair or poor self-rated health status among adults
0.5785373390851917  All teeth lost among adults aged >=65 years
0.5425489982528412  Vision disability among adults
0.5457473803776466  Cognitive disability among adults
0.453431907377555  Frequent mental distress among adults
0.3152132415829904  Lack of social and emotional support among adults
0.5846167646592714  Mobility disability among adults
0.4320945944352093  Current asthma among adults
0.5515245880466111  Chronic obstructive pulmonary disease among adults
0.4381443387324863  Coronary heart disease among adults
0.5313180859113242  Diagnosed diabetes among adults
0.4244206895598808  Hearing disability among adults
0.266203498242165  Depression among adults
0.4454247339707587  Short sleep duration among adults
0.5530376354779466  Obesity among adults
